# Using Azure Open AI with LangChain

## In these examples, we use LangChain to connect to three different kinds of sources
1. an API, by providing the LLM with documentation on the API
2. a Database, here a local sqlite instance, but demonstrating the generation of SQL On the fly.
3. a CSV file, here using an agent to query information and return a chain of thought and response

> ### your .env file should have the following 
``` 
OPENAI_API_TYPE="azure"
OPENAI_API_VERSION="2022-12-01"
OPENAI_API_KEY="<YOUR AZURE OPEN AI KEY>"
OPENAI_API_BASE="https://<YOUR BASE URL>.openai.azure.com/"
```

## Querying an API with an Azure Open AI text completion model 

Import the Langchain modules

In [7]:
#%pip install -r ./common/requirements.txt
#%pip install python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.chains.api.prompt import API_RESPONSE_PROMPT

In [2]:
from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate
from langchain.llms import AzureOpenAI

Load the .env file

In [3]:
from dotenv import load_dotenv
import os

In [4]:
load_dotenv()

True

Use the deployment and model names that you have already setup in the Azure Open AI portal

In [6]:
llm = AzureOpenAI(
    deployment_name="davinci",
    model_name="text-davinci-003", 
)

In [5]:
llm = AzureOpenAI(
    deployment_name="gpt4",
    model_name="gpt-4", 
)

c:\source\.conda\lib\site-packages\langchain\llms\openai.py:171: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\source\.conda\lib\site-packages\langchain\llms\openai.py:716: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


Test the LLM works

In [7]:
llm("Tell me a joke")

"\n\nQ: What did one wall say to the other wall?\nA: I'll meet you at the corner!"

Now we use the API Chain method from Langchain. This points the LLM to go read the documentation for an API and then construct the API call based on the question. This is how the LLM knows what fields are available to be queried. 

In [8]:
from langchain.chains.api import open_meteo_docs
chain_new = APIChain.from_llm_and_api_docs(llm, open_meteo_docs.OPEN_METEO_DOCS, verbose=True)

Ask a question in normal english and watch how the LLM uses the API to figure out what the Lat/Long for your given location is, and then construct an API call with a bunch of parameter. The API returns the JSON which is then converted back into normal english for the user. 

In [9]:
chain_new.run('What is the weather like right now in Sydney, Australia in degrees centigrade?')



> Entering new APIChain chain...


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


 https://api.open-meteo.com/v1/forecast?latitude=-33.8688&longitude=151.2093&current_weather=true&temperature_unit=celsius
{"latitude":-33.75,"longitude":151.125,"generationtime_ms":0.03790855407714844,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":86.0,"current_weather":{"temperature":24.1,"windspeed":22.6,"winddirection":343,"weathercode":3,"is_day":1,"time":"2023-10-03T22:00"}}

> Finished chain.


' The current weather in Sydney, Australia is 24.1 degrees Celsius with a windspeed of 22.6 km/h and a winddirection of 343 degrees.'

***